In [33]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import apimoex

import plotly.graph_objects as go
import datetime

In [2]:
def get_data_ticker_yfinance(ticker_name, period = '1y'):
    
    """
    получение данных по тикеру с Yahoo!
    """
    ticker = yf.Ticker(ticker_name)

    data = ticker.history(period = period, interval='1d')
    
    return data
    

In [3]:
def get_data_ticker_moex(ticker_name, start=None, end=None):
    """
    
    """
    with requests.Session() as session:
        try:
            data = apimoex.get_board_history(session, ticker_name, start=start, end=end)
            df = pd.DataFrame(data)
            df.set_index('TRADEDATE', inplace=True)
        except:
            df = pd.DataFrame()
    return df



In [4]:
def get_data_ticker(ticker_name, period):
    """
    
    """
    
    dict_periods = {"1d":1,"5d":5,"1mo":30,"3mo":3*30,"6mo":6*30,"1y":365,"2y":2*365,"5y":5*365,"10y":10*365,"max":None}

    if period != "max":
        days = dict_periods[period]
        end = datetime.datetime.today()
        start = end - datetime.timedelta(days = days)
        end_str = end.strftime("%Y-%m-%d")
        start_str = start.strftime("%Y-%m-%d")
    else: 
        end_str = None
        start_str = None


    data = get_data_ticker_moex(ticker_name, start = start_str, end = end_str).rename(columns={'CLOSE':'Close'})
    
    if data.shape[0]==0:
        data = get_data_ticker_yfinance(ticker_name, period = period)
        
    return data


In [85]:
def sma_calc(data, short = 5, medium = 30, long = 90):

    dict_periods = {'SMA_short':short, 
                   'SMA_medium':medium,
                   'SMA_long':long}

    cols = ['Close', 'SMA_short', 'SMA_medium', 'SMA_long']
    data['SMA_short'] = data['Close'].rolling(short, ).mean()
    data['SMA_medium'] = data['Close'].rolling(medium).mean()
    data['SMA_long'] = data['Close'].rolling(long).mean()
    data['Diff'] = data.SMA_medium - data.SMA_short
    data['Cross'] = np.select([((data.Diff < 0) & (data.Diff.shift() > 0)), ((data.Diff > 0) & (data.Diff.shift() < 0))], ['Up', 'Down'], None)
    fig = go.Figure()
    for col in cols:
        if col in dict_periods:
            period = dict_periods[col]
            name = f"{col[:3]} - {period} d"
        else: 
            name = col

        fig.add_trace(go.Scatter(x=data.index, y=data[col], name = name))
    
    if data[data['Cross'].notnull()]['Cross'].shape[0]>=1:
        for cross_cat in data[data['Cross'].notnull()]['Cross'].unique():
            if cross_cat == 'Down':
                color = 'red'
            else: 
                color = 'green'
            fig.add_trace(go.Scatter(x=data[data['Cross']==cross_cat].index, y=data[data['Cross']==cross_cat]['SMA_short'], 
                                     name = cross_cat, mode = 'markers',
                                    opacity=0.5,
                                    marker=dict(
                                        color=color,
                                        size=10,
                                        line=dict(
                                            color=color,
                                            width=8
                                        ))))

    fig.update_layout(title = f'<b>{ticker_name}</b>', width = 700, height = 500)
    fig.show()
    return data

In [92]:
ticker_name = 'NEE'
period = '2y'

In [93]:
data = get_data_ticker(ticker_name, period)

data = sma_calc(data)

In [94]:
data[data['Cross'].notnull()]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_short,SMA_medium,SMA_long,Diff,Cross
Date,,,,,,,,,,,,
2019-04-05,44.842335,45.069646,44.515570,45.008083,6452800,0.00,0.0,44.883057,44.994284,NaN,0.111228,Down
2019-04-29,45.469812,45.675813,45.133578,45.408249,7094400,0.00,0.0,45.169565,45.165540,NaN,-0.004024,Up
2019-05-08,44.894422,44.991502,44.349816,44.487152,7887200,0.00,0.0,45.115107,45.139336,NaN,0.024229,Down
2019-05-13,45.325369,46.270142,45.244863,46.156483,10632800,0.00,0.0,45.127892,45.108870,NaN,-0.019022,Up
2019-11-06,54.513377,54.752997,54.297722,54.510983,6976400,0.00,0.0,55.507795,55.649329,52.912630,0.141534,Down
2019-11-20,55.912746,56.732244,55.764184,56.363232,8506000,0.00,0.0,55.655399,55.354597,53.456742,-0.300801,Up
2020-02-28,60.091791,61.486630,58.864046,61.208145,27474800,0.00,0.0,63.839078,64.425825,59.259208,0.586747,Down
2020-03-05,67.090197,67.770664,65.973847,66.874680,12412000,0.00,0.0,65.123381,65.029061,59.687498,-0.094320,Up
2020-03-11,60.346061,60.350905,57.769488,58.423317,19631200,0.00,0.0,63.264082,64.861391,59.961300,1.597309,Down


In [95]:
dict_periods = {"1d":1,"5d":5,"1mo":30,"3mo":3*30,"6mo":6*30,"1y":365,"2y":2*365,"5y":5*365,"10y":10*365,"max":None}

In [97]:
dict_periods.keys()

dict_keys(['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'max'])